In [85]:
#Importing dependancies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import psycopg2 as ps
from psycopg2 import sql, connect
from getpass import getpass

In [86]:
#Importing Sklearn
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
#Connecting to AWS
import getpass
engine = ps.connect(
    database="postgres",
    user= input("Username: "),
    password= getpass.getpass("Password: "),
    host="highered-db.cx2ll8zuuepz.us-east-2.rds.amazonaws.com",
    port='5432'
)

In [594]:
cr = engine.cursor()
cr.execute('SELECT * FROM highered_data_table;')
tmp = cr.fetchall()

In [595]:
#Converting to Pandas DataFrame
df = pd.DataFrame(tmp, columns= ['Year','UNITID','FACSTAT','ARANK','HRTOTLT','HRTOTLM','HRTOTLW','INSTNM','CITY','STABBR','ZIP',
                                    'CONTROL','HLOFFER','INSTSIZE','LONGITUD','LATITUDE'])
df

,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,0,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,10,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,10,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,10,2,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,10,3,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192599,2020,232618,0,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192600,2020,232618,10,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192601,2020,232618,10,6,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192602,2020,232618,40,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100


In [87]:
#Loading the cleaned Data
file_path = Path('highered_merged_data.csv')
df = pd.read_csv(file_path)
df.head(5)



c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,0,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,10,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,10,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,10,2,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,10,3,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368


In [88]:
# df_test = df.drop_duplicates()
df_test = df.drop(['FACSTAT', 'ARANK'], axis=1)
df_test.sample(10)

,Year,UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
45894,2019,217989,23,12,11,Denmark Technical College,Denmark,SC,29042,1,3,1,-81.123629,33.313355
155385,2021,177816,2,2,0,Kenrick Glennon Seminary,Saint Louis,MO,63119,2,7,1,-90.324024,38.584608
97458,2020,192688,13,7,6,Mandl School-The College of Allied Health,New York,NY,10019,3,3,1,-73.983786,40.764404
89924,2020,173896,2,1,1,Luther Seminary,Saint Paul,MN,55108,2,9,1,-93.196422,44.984629
158895,2021,187897,13,8,5,New Mexico Highlands University,Las Vegas,NM,87701,1,7,2,-105.219197,35.596233
61108,2019,446640,45,30,15,Harrisburg University of Science and Technology,Harrisburg,PA,17101-2208,2,9,2,-76.880261,40.261930
34756,2019,196185,93,49,44,SUNY Oneonta,Oneonta,NY,13820-4015,1,8,3,-75.063632,42.468861
171282,2021,212878,3,2,1,Harrisburg Area Community College,Harrisburg,PA,17110,1,3,4,-76.886408,40.296777
122111,2020,365204,91,21,70,Vista College,El Paso,TX,79925,3,3,2,-106.411952,31.786912
121590,2020,243780,3,2,1,Purdue University-Main Campus,West Lafayette,IN,47907,1,9,5,-86.914435,40.428206


In [89]:
#Dropping duplicates
df_test.drop_duplicates(inplace=True)

In [90]:
df_test1 = df_test.groupby(['UNITID', 'Year'])['HRTOTLT'].sum().unstack(fill_value=0)
#.unstack(fill_value=0)
df_test1.head(10)

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
100751,6746,6708,6928
100760,137,127,119
100812,274,256,292
100830,865,1011,1004


In [91]:
#Creating dataframe with all other info
df_test2 = df_test[['UNITID','INSTNM','CITY','STABBR','ZIP','CONTROL','HLOFFER','INSTSIZE','LONGITUD','LATITUDE']]

In [92]:
#Dropping duplicates from df2
df_test2.drop_duplicates(inplace=True)
df_test2.sample(5)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,UNITID,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
102388,200484,Trinity Bible College and Graduate School,Ellendale,ND,58436,2,9,1,-98.517906,46.001037
24577,172334,Spring Arbor University,Spring Arbor,MI,49283-9799,2,7,2,-84.556074,42.206568
20804,163657,Prince George's Community College,Largo,MD,20774-2199,1,3,4,-76.824623,38.888689
74610,136330,Palm Beach Atlantic University,West Palm Beach,FL,33401,2,9,2,-80.052470,26.704400
53868,236504,South Seattle College,Seattle,WA,98106-1499,1,5,3,-122.352672,47.547500


In [93]:
df_test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 100654 to 497286
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2019    3756 non-null   int64
 1   2020    3756 non-null   int64
 2   2021    3756 non-null   int64
dtypes: int64(3)
memory usage: 117.4 KB


In [94]:
df_test1.dropna()

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
...,...,...,...
496681,0,0,10
496991,0,0,7
497046,0,0,112


In [95]:
#Using drop() to delete rows based on column value
df_test1.drop(df_test1[df_test1[2019] == 0].index, inplace = True)

In [96]:
df3 = pd.merge(left=df_test1, right=df_test2, left_on='UNITID', right_on='UNITID')
df3.shape
df3.sample(5)

,UNITID,2019,2020,2021,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
6848,443748,9,15,28,Altierus Career College-Norcross,Norcross,GA,30093,2,3,1,-84.155475,33.934121
1529,145372,252,200,185,Greenville University,Greenville,IL,62246,2,7,1,-89.408240,38.894162
5912,236610,258,260,238,Shoreline Community College,Shoreline,WA,98133,1,3,2,-122.358626,47.747402
2122,157216,431,385,410,Lindsey Wilson College,Columbia,KY,42728-1223,2,9,2,-85.299997,37.101502
3879,196653,178,162,111,Trocaire College,Buffalo,NY,14220-2094,2,5,2,-78.812398,42.846693


In [97]:
#Calculating retention rate
df3["Retention_Rate"] = (df3[2021]/df3[2019] * 100).round(2).astype(float)
df3.sample(10)

,UNITID,2019,2020,2021,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,Retention_Rate
3713,193830,1307,1192,1293,New York Medical College,Valhalla,NY,10595,2,9,2,-73.810042,41.085031,98.93
7569,491808,4,4,4,Meridian University,Petaluma,CA,94952-3092,3,9,1,-122.640029,38.230878,100.00
5101,218821,66,74,82,Spartanburg Methodist College,Spartanburg,SC,29301,2,5,2,-81.970863,34.955117,124.24
5270,221768,986,1011,1174,The University of Tennessee-Martin,Martin,TN,38238,1,7,3,-88.864211,36.340338,119.07
798,123651,335,328,328,Vanguard University of Southern California,Costa Mesa,CA,92626,2,8,2,-117.900911,33.662686,97.91
2381,164173,457,457,468,Stevenson University,Stevenson,MD,21153,2,7,2,-76.702092,39.421988,102.41
829,124706,28,26,26,Concorde Career College-San Bernardino,San Bernardino,CA,92408,3,4,1,-117.283995,34.066823,92.86
945,127909,97,98,95,Red Rocks Community College,Lakewood,CO,80228,1,7,3,-105.150094,39.720277,97.94
7581,492801,20,22,10,Drury University-College of Continuing Profess...,Springfield,MO,65802,2,5,2,-93.286693,37.218523,50.00
349,110486,998,1095,926,California State University-Bakersfield,Bakersfield,CA,93311,1,9,4,-119.104711,35.350005,92.79


In [98]:
#Adding retention boolean
df3["Retention"] = np.where(df3['Retention_Rate'] > 99.99, True, False)
df3

,UNITID,2019,2020,2021,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,Retention_Rate,Retention
0,100654,874,1176,1027,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368,117.51,True
1,100654,874,1176,1027,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368,117.51,True
2,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294-0110,1,9,5,-86.799345,33.505697,103.38,True
3,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294,1,9,5,-86.799345,33.505697,103.38,True
4,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294,1,9,5,-86.799345,33.505697,103.38,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7603,494737,19,19,19,Yeshiva Yesoda Hatorah Vetz Chaim,Brooklyn,NY,11211,2,5,1,-73.961026,40.706959,100.00,True
7604,494843,20,16,16,Fortis College-Landover,Landover,MD,20785-2223,3,3,1,-76.866209,38.949983,80.00,False
7605,494843,20,16,16,Fortis College-Landover,Landover,MD,20785,3,3,1,-76.866209,38.949983,80.00,False
7606,494852,25,28,26,Stautzenberger College-Rockford Career College,Rockford,IL,61108-3900,3,3,1,-89.027291,42.255903,104.00,True


In [99]:
#Dropping Retention Rate Column
df3.drop(columns=["Retention"])

,UNITID,2019,2020,2021,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,Retention_Rate
0,100654,874,1176,1027,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368,117.51
1,100654,874,1176,1027,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368,117.51
2,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294-0110,1,9,5,-86.799345,33.505697,103.38
3,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294,1,9,5,-86.799345,33.505697,103.38
4,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294,1,9,5,-86.799345,33.505697,103.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7603,494737,19,19,19,Yeshiva Yesoda Hatorah Vetz Chaim,Brooklyn,NY,11211,2,5,1,-73.961026,40.706959,100.00
7604,494843,20,16,16,Fortis College-Landover,Landover,MD,20785-2223,3,3,1,-76.866209,38.949983,80.00
7605,494843,20,16,16,Fortis College-Landover,Landover,MD,20785,3,3,1,-76.866209,38.949983,80.00
7606,494852,25,28,26,Stautzenberger College-Rockford Career College,Rockford,IL,61108-3900,3,3,1,-89.027291,42.255903,104.00


In [100]:
#Creating feature
X = df3.drop("Retention", axis=1)
X = X.drop("Retention_Rate", axis=1)
X = pd.get_dummies(X)

In [101]:
#Adding a target thought? 
target = ["Retention"]

In [102]:
#Creating a target
y = df3.loc[:, target].copy()

In [103]:
#Decribing
X.describe()

,UNITID,2019,2020,2021,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,INSTNM_A T Still University of Health Sciences,...,ZIP_99258-0001,ZIP_99301,ZIP_99324-1198,ZIP_99362,ZIP_99362-9267,ZIP_99507-1033,ZIP_99508,ZIP_99723,ZIP_99775,ZIP_99801-8697
count,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,...,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000,7608.000000
mean,223034.281020,785.875789,770.062040,770.741588,1.692429,6.085568,2.167061,-90.374060,37.568760,0.000263,...,0.000131,0.000131,0.000131,0.000131,0.000131,0.000131,0.000263,0.000131,0.000131,0.000131
std,107304.474205,2089.264711,2065.013648,2110.507128,0.703826,2.425037,1.148880,18.933863,5.889101,0.016213,...,0.011465,0.011465,0.011465,0.011465,0.011465,0.011465,0.016213,0.011465,0.011465,0.011465
min,100654.000000,1.000000,0.000000,0.000000,1.000000,3.000000,-2.000000,-170.742774,-14.322636,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,153977.000000,52.000000,48.000000,48.000000,1.000000,3.000000,1.000000,-97.471508,34.061409,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,195997.000000,216.500000,205.500000,204.000000,2.000000,7.000000,2.000000,-86.511843,38.761995,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,232113.250000,602.750000,585.250000,588.000000,2.000000,9.000000,3.000000,-78.783365,41.529822,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,494852.000000,36293.000000,30347.000000,35552.000000,3.000000,9.000000,5.000000,171.378129,71.324702,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(Counter(y_train['Retention']))
print(Counter(y_test['Retention']))

Counter({False: 3335, True: 2371})
Counter({False: 1083, True: 819})


In [105]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
print(Counter(y_train['Retention']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Counter({False: 3335, True: 2371})


In [106]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.89660893123497

In [107]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[986,  97],
       [ 96, 723]], dtype=int64)

In [108]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.91      0.91      0.88      0.91      0.90      0.81      1083
       True       0.88      0.88      0.91      0.88      0.90      0.80       819

avg / total       0.90      0.90      0.89      0.90      0.90      0.80      1902



In [44]:
#***** The below blocks are the Easy Ensemble Model, this took over 175 minutes to run *****

In [45]:
# # Train the EasyEnsembleClassifier
# from imblearn.ensemble import EasyEnsembleClassifier
# ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
# ee_model.fit(X_train, y_train)
# print(Counter(y_train['Retention']))

In [46]:
# # Calculated the balanced accuracy score
# y_pred = ee_model.predict(X_test)
# balanced_accuracy_score(y_test,y_pred)

In [47]:
# # Display the confusion matrix
# cm2 = confusion_matrix(y_test, y_pred)
# cm2